In [10]:
from openai import OpenAI

client = OpenAI(base_url="http://172.27.21.160:11434/v1", api_key="ollama")

response = client.chat.completions.create(
    model="llama3.1:8b",
    messages=[{"role": "user", "content": "Well hello there"}]
)

print(response.choices[0].message.content)

Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?


In [1]:
import dspy
print(hasattr(dspy, "retrieve"))


False


In [2]:
import dspy

lm = dspy.LM(
    "openai/llama3.1:8b",  # model name is arbitrary but should match your endpoint
    api_base="http://172.27.21.160:11434/v1",  # your local server
    api_key="ollama",  # if your server requires this token
    model_type="chat"  # use 'chat' for instruct-style models
)

dspy.configure(lm=lm)

In [3]:
print(lm(messages=[{"role": "user", "content": "What is the capital of France?"}]))

['The capital of France is Paris.']


In [4]:
qa = dspy.ChainOfThought("question -> answer")
response = qa(question="Explain how brk() and sbrk() differ in Unix memory management.")
print(response.answer)

The key difference between brk() and sbrk() is that brk() sets the break point, which determines the maximum amount of memory that can be allocated by a process, whereas sbrk() allocates or deallocates memory within the existing heap.


In [5]:
from pydantic import BaseModel

class Date(BaseModel):
    # Somehow LLM is bad at specifying `datetime.datetime`, so
    # we define a custom class to represent the date.
    year: int
    month: int
    day: int
    hour: int

class UserProfile(BaseModel):
    user_id: str
    name: str
    email: str

class Flight(BaseModel):
    flight_id: str
    date_time: Date
    origin: str
    destination: str
    duration: float
    price: float

class Itinerary(BaseModel):
    confirmation_number: str
    user_profile: UserProfile
    flight: Flight

class Ticket(BaseModel):
    user_request: str
    user_profile: UserProfile

In [ ]:
user_database = {
    "Adam": UserProfile(user_id="1", name="Adam", email="adam@gmail.com"),
    "Bob": UserProfile(user_id="2", name="Bob", email="bob@gmail.com"),
    "Chelsie": UserProfile(user_id="3", name="Chelsie", email="chelsie@gmail.com"),
    "David": UserProfile(user_id="4", name="David", email="david@gmail.com"),
}

flight_database = {
    "DA123": Flight(
        flight_id="DA123",  # DSPy Airline 123
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=1),
        duration=3,
        price=200,
    ),
    "DA125": Flight(
        flight_id="DA125",
        origin="SFO",
        destination="JFK",
        date_time=Date(year=2025, month=9, day=1, hour=7),
        duration=9,
        price=500,
    ),
    "DA456": Flight(
        flight_id="DA456",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=1),
        duration=2,
        price=100,
    ),
    "DA460": Flight(
        flight_id="DA460",
        origin="SFO",
        destination="SNA",
        date_time=Date(year=2025, month=10, day=1, hour=9),
        duration=2,
        price=120,
    ),
}

itinery_database = {}
ticket_database = {}

In [ ]:
import random
import string


def fetch_flight_info(date: Date, origin: str, destination: str):
    """Fetch flight information from origin to destination on the given date"""
    flights = []

    for flight_id, flight in flight_database.items():
        if (
            flight.date_time.year == date.year
            and flight.date_time.month == date.month
            and flight.date_time.day == date.day
            and flight.origin == origin
            and flight.destination == destination
        ):
            flights.append(flight)
    if len(flights) == 0:
        raise ValueError("No matching flight found!")
    return flights


def fetch_itinerary(confirmation_number: str):
    """Fetch a booked itinerary information from database"""
    return itinery_database.get(confirmation_number)


def pick_flight(flights: list[Flight]):
    """Pick up the best flight that matches users' request. we pick the shortest, and cheaper one on ties."""
    sorted_flights = sorted(
        flights,
        key=lambda x: (
            x.get("duration") if isinstance(x, dict) else x.duration,
            x.get("price") if isinstance(x, dict) else x.price,
        ),
    )
    return sorted_flights[0]


def _generate_id(length=8):
    chars = string.ascii_lowercase + string.digits
    return "".join(random.choices(chars, k=length))


def book_flight(flight: Flight, user_profile: UserProfile):
    """Book a flight on behalf of the user."""
    confirmation_number = _generate_id()
    while confirmation_number in itinery_database:
        confirmation_number = _generate_id()
    itinery_database[confirmation_number] = Itinerary(
        confirmation_number=confirmation_number,
        user_profile=user_profile,
        flight=flight,
    )
    return confirmation_number, itinery_database[confirmation_number]


def cancel_itinerary(confirmation_number: str, user_profile: UserProfile):
    """Cancel an itinerary on behalf of the user."""
    if confirmation_number in itinery_database:
        del itinery_database[confirmation_number]
        return
    raise ValueError("Cannot find the itinerary, please check your confirmation number.")


def get_user_info(name: str):
    """Fetch the user profile from database with given name."""
    return user_database.get(name)


def file_ticket(user_request: str, user_profile: UserProfile):
    """File a customer support ticket if this is something the agent cannot handle."""
    ticket_id = _generate_id(length=6)
    ticket_database[ticket_id] = Ticket(
        user_request=user_request,
        user_profile=user_profile,
    )
    return ticket_id


In [ ]:
import dspy

class DSPyAirlineCustomerService(dspy.Signature):
    """You are an airline customer service agent that helps user book and manage flights.

    You are given a list of tools to handle user request, and you should decide the right tool to use in order to
    fulfill users' request."""

    user_request: str = dspy.InputField()
    process_result: str = dspy.OutputField(
        desc=(
                "Message that summarizes the process result, and the information users need, e.g., the "
                "confirmation_number if a new flight is booked."
            )
        )

In [ ]:
agent = dspy.ReAct(
    DSPyAirlineCustomerService,
    tools = [
        fetch_flight_info,
        fetch_itinerary,
        pick_flight,
        book_flight,
        cancel_itinerary,
        get_user_info,
        file_ticket,
    ]
)

In [ ]:
result = agent(user_request="please help me book a flight from SFO to JFK on 09/01/2025, my name is Adam")
print(result)

Prediction(
    trajectory={'thought_0': "To book a flight from SFO to JFK on 09/01/2025, we need to first find available flights. Let's start by fetching flight information.", 'tool_name_0': 'fetch_flight_info', 'tool_args_0': {'date': {'year': 2025, 'month': 9, 'day': 1, 'hour': 0}, 'origin': 'SFO', 'destination': 'JFK'}, 'observation_0': [Flight(flight_id='DA123', date_time=Date(year=2025, month=9, day=1, hour=1), origin='SFO', destination='JFK', duration=3.0, price=200.0), Flight(flight_id='DA125', date_time=Date(year=2025, month=9, day=1, hour=7), origin='SFO', destination='JFK', duration=9.0, price=500.0)], 'thought_1': "Now that we have fetched available flights, let's pick the best one based on duration and price.", 'tool_name_1': 'pick_flight', 'tool_args_1': {'flights': [{'flight_id': 'DA123', 'date_time': {'year': 2025, 'month': 9, 'day': 1, 'hour': 1}, 'origin': 'SFO', 'destination': 'JFK', 'duration': 3.0, 'price': 200.0}, {'flight_id': 'DA125', 'date_time': {'year': 2025,

In [ ]:
toxicity = dspy.Predict(
    dspy.Signature(
        "comment -> toxic: bool",
        instructions="Mark as 'toxic' if the comment includes insults, harassment, or sarcastic derogatory remarks.",
    )
)
comment = "you are beautiful."
toxicity(comment=comment).toxic

False

In [ ]:
question = "What's something great about the ColBERT retrieval model?"

# 1) Declare with a signature, and pass some config.
classify = dspy.ChainOfThought('question -> answer', n=5)

# 2) Call with input argument.
response = classify(question=question)

# 3) Access the outputs.
response.completions.answer

['ColBERT is effective because it captures contextual information through pre-trained language models like BERT, enabling accurate and relevant retrievals.']

In [ ]:
print(f"Reasoning: {response.reasoning}")
print(f"Answer: {response.answer}")

Reasoning: ColBERT is a state-of-the-art retrieval model that excels in document retrieval tasks. One of its greatest strengths is its ability to effectively capture contextual information from long documents, allowing it to better understand the relevance and accuracy of retrieved results.

ColBERT's success can be attributed to several factors:

1. **Contextualized Embeddings**: ColBERT uses pre-trained language models like BERT to generate contextualized embeddings for query and document pairs. This enables the model to capture nuanced relationships between words within a sentence or paragraph.
2. **Exact-Term Matching**: Unlike other retrieval models that rely on heuristic matching strategies, ColBERT takes an exact-term matching approach, which results in more accurate and relevant retrievals.
3. **Incorporating BERT's Attention Mechanism**: ColBERT's design incorporates the attention mechanism from BERT to weigh the importance of different words within a document when generating 

In [ ]:
def search(query: str) -> list[str]:
    """Retrieves abstracts from Wikipedia."""
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

rag = dspy.ChainOfThought('context, question -> response')

question = "What's the name of the castle that David Gregory inherited?"
ans=rag(context=search(question), question=question)
print(ans.reasoning)

The question asks about the castle inherited by David Gregory. The provided text mentions that David Gregory "inherited Kinnairdy Castle in 1664." Therefore, the answer to the question is the name of the castle mentioned.


In [ ]:
def evaluate_math(expression: str) -> float:
    return dspy.PythonInterpreter({}).execute(expression)

def search_wikipedia(query: str) -> str:
    results = dspy.ColBERTv2(url='http://20.102.90.50:2017/wiki17_abstracts')(query, k=3)
    return [x['text'] for x in results]

react = dspy.ReAct("question -> answer: float", tools=[evaluate_math, search_wikipedia])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)


1715.0


In [9]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from pptx.enum.shapes import MSO_SHAPE

# Create presentation
prs = Presentation()
prs.slide_width = Inches(13.33)
prs.slide_height = Inches(7.5)

# Define font styles
def format_textbox(textbox, font_size=Pt(20), bold=False):
    for paragraph in textbox.text_frame.paragraphs:
        # Ensure there's at least one run to avoid IndexError when runs is empty
        if paragraph.runs:
            run = paragraph.runs[0]
        else:
            run = paragraph.add_run()
        run.font.name = 'Times New Roman'
        run.font.size = font_size
        run.font.bold = bold
        paragraph.alignment = PP_ALIGN.LEFT

# Title Slide
slide = prs.slides.add_slide(prs.slide_layouts[0])
slide.shapes.title.text = "LLM-Driven Halide Code Generation and Automated Validation"
slide.placeholders[1].text = "Project Overview"
format_textbox(slide.shapes.title, Pt(36), True)
format_textbox(slide.placeholders[1], Pt(24))

# Slide 1 — Project Overview
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Project Overview"
content = slide.placeholders[1]
content.text = "Goal → Use large language models to generate correct Halide (Python) image-processing programs."
for bullet in [
    "Auto-validate results on numerical & image-based test cases.",
    "Feedback loop: Model → Code → Run → Validator → Feedback → Improved Code."
]:
    p = content.text_frame.add_paragraph()
    p.text = bullet
format_textbox(content, Pt(20))

# Slide 2 — System Architecture
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "System Architecture"
content = slide.placeholders[1]
content.text = "Flow Diagram:"
format_textbox(content, Pt(20), True)

# Add flow diagram
steps = [
    "User Prompt",
    "HalidePipeline\n(LLM + reasoning)",
    "Halide Code + Test Cases (JSON)",
    "HalideValidatorLoop",
    "Execution on test images",
    "Pass/Fail Feedback → LLM"
]
left = Inches(0.5)
top = Inches(2)
width = Inches(2.2)
height = Inches(0.8)
for i, step in enumerate(steps):
    box = slide.shapes.add_shape(MSO_SHAPE.ROUNDED_RECTANGLE, left + Inches(i * 2), top, width, height)
    box.text = step
    box.fill.solid()
    box.fill.fore_color.rgb = RGBColor(240, 240, 255)
    box.line.color.rgb = RGBColor(0, 0, 0)
    format_textbox(box, Pt(14))

# Add module list
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Key Modules"
content = slide.placeholders[1]
for bullet in [
    "HalidePipeline (Chain-of-Thought reasoning)",
    "HalideValidatorLoop (execution + comparison)",
    "React loop for adaptive correction"
]:
    p = content.text_frame.add_paragraph()
    p.text = bullet
format_textbox(content, Pt(20))

# Slide 3 — Validation Process
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Validation Process"
content = slide.placeholders[1]
for bullet in [
    "Each round:",
    "• Generate code + test cases",
    "• Save input.png and expected_output.png",
    "• Run Halide code → output.png",
    "• Compare pixel-wise (mean & max diff)",
    "• Pass/fail feedback sent back",
    "",
    "Example metrics:",
    "• Expected mean: 0.85",
    "• Actual mean:   0.846",
    "• Max diff: 0.078"
]:
    p = content.text_frame.add_paragraph()
    p.text = bullet
format_textbox(content, Pt(20))

# Slide 4 — Key Findings & Challenges
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Key Findings & Challenges"
table_data = [
    ("Successes", "Challenges"),
    ("LLM produced valid Halide code for multiple ops", "Occasional truncation & syntax drift"),
    ("Auto-execution validated correctness", "JSON outputs sometimes malformed"),
    ("Feedback loop improved reliability", "Unclamped expected values caused test mismatch"),
    ("Test-image pipeline proved useful", "LLM sometimes switched to C++")
]
rows, cols = len(table_data), len(table_data[0])
table = slide.shapes.add_table(rows, cols, Inches(0.5), Inches(1.5), Inches(12), Inches(3)).table
for r in range(rows):
    for c in range(cols):
        cell = table.cell(r, c)
        cell.text = table_data[r][c]
        cell.text_frame.paragraphs[0].font.name = 'Times New Roman'
        cell.text_frame.paragraphs[0].font.size = Pt(16)
        cell.text_frame.paragraphs[0].font.bold = r == 0

# Slide 5 — Future Work
slide = prs.slides.add_slide(prs.slide_layouts[1])
slide.shapes.title.text = "Future Work"
content = slide.placeholders[1]
for bullet in [
    "Add scheduling optimizations (Halide vectorize, tile, parallel) to test performance.",
    "Introduce real-image benchmarks (blur, sharpen, tone-map).",
    "Explore multi-agent feedback: validator + static analyzer.",
    "Deploy pipeline as a web demo with visual diff viewer."
]:
    p = content.text_frame.add_paragraph()
    p.text = bullet
format_textbox(content, Pt(20))

# Save presentation
prs.save("Halide_LLM_Research_Presentation.pptx")
